## Generating fixtures to test the Equation Of Time functions, the Longitude Fitting Functions and the Longitude Calculation Functions

### Expected Longitude Output is downloaded directly from https://maps.nrel.gov/pvdaq/ PVDAQ Contributed Sites

### Expected longitude = -76.6636

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
# PVInsight Code Imports
from solardatatools import DataHandler
from solardatatools.dataio import get_pvdaq_data
path = Path.cwd().parent.parent.parent
os.chdir(path)

### Load data table from external source

#### For today's example, we're loading data from NREL's PVDAQ API, which is a publically available PV generatation data set.

In [3]:
data_frame = get_pvdaq_data(sysid=1199, year=[2015, 2016, 2017], api_key='DEMO_KEY')

CAUTION: Multiple scan rates detected!=======================] 100.0% ...queries complete in 24.4 seconds       
Scan rates (in seconds): [300, 280]
0 transitions detected.
Suggest splitting data set between:




In [4]:
gmt_offset = -5

### Running the  DataHandler

In [5]:
dh = DataHandler(data_frame[0])
dh.fix_dst()

In [6]:
dh.run_pipeline(power_col='ac_power', fix_shifts=False, correct_tz=False)

total time: 16.84 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              5.41s
Cleaning                   0.36s
Filtering/Summarizing      11.06s
    Data quality           0.49s
    Clear day detect       0.30s
    Clipping detect        8.89s
    Capacity change detect 1.38s



In [7]:
day_of_year = dh.day_index.dayofyear

In [8]:
day_range = np.ones(day_of_year.shape, dtype=bool)

In [9]:
daily_meas = dh.filled_data_matrix.shape[0]

In [37]:
daytime_threshold = 0.001

In [38]:
from pvsystemprofiler.algorithms.performance_model_estimation import find_fit_costheta

In [39]:
scale_factor_costheta, costheta_fit = find_fit_costheta(dh.filled_data_matrix, dh.daily_flags.no_errors)

In [40]:
from pvsystemprofiler.algorithms.tilt_azimuth.daytime_threshold_quantile import filter_data

In [41]:
from pvsystemprofiler.algorithms.latitude.hours_daylight import calculate_hours_daylight

In [42]:
hours_daylight = calculate_hours_daylight(dh.filled_data_matrix, threshold=daytime_threshold)

In [43]:
boolean_filter = filter_data(dh.filled_data_matrix, daytime_threshold=daytime_threshold, x1=0.9, x2=0.9)

In [44]:
from pvsystemprofiler.utilities.hour_angle_equation import calculate_omega

In [45]:
latitude = 39.4856
longitude = -76.6636
tilt = 180
azimuth = 20
gmt_offset = -5

In [46]:
omega = calculate_omega(dh.data_sampling, dh.num_days, longitude, day_of_year, gmt_offset)

In [47]:
from pvsystemprofiler.utilities.declination_equation import delta_cooper

In [48]:
delta = delta_cooper(day_of_year, daily_meas)

In [49]:
delta_f = delta[boolean_filter]

In [50]:
omega_f = omega[boolean_filter]

In [51]:
from pvsystemprofiler.algorithms.angle_of_incidence.lambda_functions import select_function

In [52]:
func_customized, bounds = select_function(latitude, None, None)

In [53]:
from pvsystemprofiler.algorithms.angle_of_incidence.curve_fitting import run_curve_fit

In [54]:
estimates = run_curve_fit(func=func_customized, keys=['tilt', 'azimuth'], delta=delta_f, omega=omega_f,
                                  costheta=costheta_fit, boolean_filter=boolean_filter, init_values=[30, 16],
                                  fit_bounds=bounds)

In [55]:
estimates

array([  8.14058919, 178.03158547])

In [29]:
from pvsystemprofiler.estimator import ConfigurationEstimator

In [30]:
est = ConfigurationEstimator(dh, gmt_offset=-5)

In [31]:
est.estimate_orientation(longitude=longitude, latitude=latitude)

In [32]:
pwd

'/Users/londonoh/Documents/github/pv-system-profiler'

In [33]:
pwd

'/Users/londonoh/Documents/github/pv-system-profiler'

In [34]:
est.tilt

37.83673399456036

In [35]:
est.azimuth

-0.9222420869548729